In [1]:
import os
import warnings
warnings.simplefilter('ignore')

import numpy as np
import random
from functools import partial

import torch
import torch.nn as nn

from predictive_model_gpu import make_fc

from greedy_coverage_gpu import set_func, marginal_vec, greedy
from greedy_submodular_new import GreedyOptimizer, GreedyContext

from IPython.lib.deepreload import reload
%load_ext autoreload


beginning

In [2]:

beta = 1
sample_size = 10
eps = 0.2
k = 5

num_items = 100
num_targets = 500

num_random_iter = 1
num_instances = 30
num_features = 43

num_epochs = 5
batch_size = 100
learning_rate = 1e-3

num_layers = 2
activation = 'relu'
intermediate_size = 200

def init_weights(m):
    if type(m) == nn.Linear:
        m.weight.data.uniform_(.0, .01)



In [3]:
# setting device on GPU if available, else CPU
device = torch.device('cpu' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()
torch.cuda.empty_cache()
#Additional Info when using cuda
if device.type == 'cuda':
    
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Using device: cpu



In [4]:

### instances ###
path = 'instances/'
Ps = torch.from_numpy(np.load(path + 'Ps.npz')['Ps']).float().to(device)      
data = torch.from_numpy(np.load(path + 'data.npz')['data']).float().to(device)
trains, tests = [], []
for i in range(num_random_iter):
    trains.append(np.load(path + '{}.npz'.format(i))['train'])
    tests.append(np.load(path + '{}.npz'.format(i))['test'])
w = np.ones(num_targets, dtype=np.float32)



In [5]:

### main ###
train_scores = []
test_scores  = []

for idx in range(num_random_iter):
    print(idx)
    test  = tests[idx] 
    train = trains[idx]
    dataset = torch.utils.data.TensorDataset(data[train], Ps[train])
    
    net = make_fc(num_features, num_layers, activation, intermediate_size)
    net.apply(init_weights)
    net = net.to(device)

    optimizer = torch.optim.Adam(net.parameters(), lr = learning_rate)

    for epoch in range(num_epochs):
        print('epoch{}'.format(epoch))
        data_loader = torch.utils.data.DataLoader(dataset, batch_size = batch_size, shuffle = True)
        
        for X_batch, P_batch in data_loader:
            loss = 0 
            for X, P in zip(X_batch, P_batch):
                X, P = X.to(device), P.to(device)
                true_set_func = partial(set_func, P = P, w = w)
                marginal_vec_pred = partial(marginal_vec, w = w)
                pred = net(X).view_as(P)  
                               
                
                loss -= GreedyOptimizer.apply(pred, true_set_func, marginal_vec_pred, num_items, k, eps,sample_size, beta)
            loss = loss / batch_size
            print(loss)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        

    def eval_grd(net, instances):
        return np.mean([set_func(greedy(k, net(data[i]).view_as(Ps[0]), w)[1], Ps[i], w) for i in instances])

    train_score = eval_grd(net, train)
    test_score  = eval_grd(net, test)

    print(train_score)
    print(test_score)

    train_scores.append(train_score)
    test_scores.append(test_score)

print(np.mean(train_scores))
print(np.mean(test_scores))



0
epoch0


In [7]:
%reset

end
